# Stochastic Variational Gaussian Process Classification

@Author Juanwu Lu

@Date   Mar-2-2023

The scalable Gaussian Process Classifier model using Stochastic Variantional inference was first introduced by J. Hensman, et al. in the paper [Scalable Variational Gaussian Process Classifier](https://arxiv.org/pdf/1411.2005.pdf).

## Preliminaries

Related topics to the proposed SVGPC model in the original papers are **Guanssian Process Classification** and **Sparse Gaussian Processes for Regression**. As mentioned in the paper introduction, although sharing some common addresses, the two area seldomly overlap.

### Gaussian Process Classification

First, let's consider the simplest case - binary classification. If denote the binary class labels by $\mathbf{y}=\left\{y_n\right\}_{n=1}^N$, and then collect the input data into a design matrix $\mathbf{X}=\left\{\mathbf{x}_n\right\}_{n=1}^N$. The covariance matrix is evaluated at all pairs of input vectors, and arrive at a zero-mean prior for the values of the GP function at the input points $p(\mathbf{f})=\mathcal{N}(\mathbf{f}\mid\mathbf{0},\mathbf{K}_{nn})$.

To perform binary classfication on the Gaussian prior, we **squash** the prior using a *sigmoidal inverse link function* $\phi(x)=\int\limits_{-\infty}^x\mathcal{N}(a\mid 0, 1)da$, and a Bernoulli likelihood $\mathscr{B}(y_n\mid\phi(f_n))=\phi(f_n)^{y_n}\left(1-\phi(f_n)\right)^{1-y_n}$ to condition the data on the transformed function values. Following the assumption that **data are independent identical distributed (i.i.d)**, we have the joint distribution given by
$$
    p(\mathbf{y},\mathbf{f})=\prod\limits_{n=1}^N\mathcal{B}(y_n\mid\phi(f_n))\mathcal{N}(\mathbf{f}\mid\mathbf{0},\mathbf{K}_{nn}).
$$

As a result, the problem will now be to learn the posterior over the function values given by $p(\mathbf{f}\mid\mathbf{y})$. The challenge is that the matrix inverse computation here often requires $\mathscr{O}(N^3)$ computation for a dataset of size $N$.

### Sparse Gaussian Processes for Regression

One way to tackle the computation burden is by inducing additional input-output paris $\mathbf{Z}$ and $\mathbf{u}$, known as 'inducing inputs' and 'inducing variables'. Then the joint distrbution of $\mathbf{f}$ and $\mathbf{u}$ is given by
$$
    p(\mathbf{f}, \mathbf{u})=\mathcal{N}\left(\begin{bmatrix}\mathbf{f} \\ \mathbf{u}\end{bmatrix}\mid\mathbf{0},\begin{bmatrix}\mathbf{K}_{nn} & \mathbf{K}_{nm} \\ \mathbf{K}_{nm}^\intercal & \mathbf{K}_{mm}\end{bmatrix}\right),
$$
where $\mathbf{K}_{mm}$ is formed by evaluting the covariance function at *all pairs of inducing input points $\mathbf{z}_m,\mathbf{z}_m$. Using the property of conditional distribution on bivariate Gaussian, we have
$$
    \mathbf{f}\mid\mathbf{u}\sim\mathcal{N}\left(\mathbf{0}+\mathbf{K}_{nm}\mathbf{K}_{mm}^{-1}(\mathbf{u}-\mathbf{0}), \mathbf{K}_{nn}-\mathbf{K}_{nm}\mathbf{K}_{mm}^{-1}\mathbf{K}_{nm}^\intercal\right),\quad \mathbf{u}\sim\mathbf{N}(0,\mathbf{K}_{mm}),
$$
and the joint distribution of $\mathbf{y}$, $\mathbf{f}$, $\mathbf{u}$ takes the form
$$
    p(\mathbf{y},\mathbf{f},\mathbf{u})=p(\mathbf{y}\mid\mathbf{f})p(\mathbf{f}\mid\mathbf{u})p(\mathbf{u}).
$$
As a result, the computation bottle neck $\mathscr{O}(N^3)$ is now reduced to $\mathscr{O}(M^3)$ where $M$ is the number of inducing points.


In [12]:
from __future__ import annotations

import math
import os
import zipfile
from urllib import request

import gpytorch
from scipy.io.arff import loadarff
import torch
import tqdm
from matplotlib import pyplot as plt

For our experiments, we test the Gaussian Process model on the [Polish Company Bankruptcy Dataset](https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data).

In [26]:
# Preprocessing the polish bankruptcy data set
if not os.path.isdir('../data/polish_bankruptcy'):
    os.makedirs('../data/polish_bankruptcy', exist_ok=True)

if not os.path.isfile('../data/polish_bankruptcy/data.zip'):
    request.urlretrieve(
        url='https://archive.ics.uci.edu/ml/' +
        'machine-learning-databases/00365/data.zip',
        filename='../data/polish_bankruptcy/data.zip'
    )

# Validate that all the dataset files exist
for i in range(1, 6):
    if not os.path.exists(f'../data/polish_bankruptcy/{i:d}year.arff'):
        with zipfile.ZipFile(
            '../data/polish_bankruptcy/data.zip', 'r'
        ) as zip_ref:
            zip_ref.extractall('../data/polish_bankruptcy/')